In [18]:
import geopandas


fires = geopandas.read_file("../Data/Facilities/DemoCairo.gpkg", layer='fire').to_crs("EPSG:3857") #Point
facilities = geopandas.read_file("../Data/Facilities/DemoCairo.gpkg", layer='facilities').to_crs("EPSG:3857") #Point
admin_poly = geopandas.read_file("../Data/Facilities/DemoCairo.gpkg", layer='NewCairoPolyDemo').to_crs("EPSG:3857") #Polygon
fires.head(50)
facilities.head(50)

fac_And_Admin = geopandas.sjoin(facilities,admin_poly[['adm3_ar', 'geometry']], how='left')
fac_And_Admin.rename_geometry('fac_geo', inplace=True)
fac_And_Admin_for_fire = fac_And_Admin.loc[fac_And_Admin['type']=='Fire']
fac_And_Admin_for_fire = fac_And_Admin.loc[fac_And_Admin['adm3_ar']=='Industrial Region East']
toto = fac_And_Admin_for_fire.to_crs("EPSG:4326")
toto['fac_lat'] = toto['fac_geo'].y
toto['fac_lon'] = toto['fac_geo'].x
geometry = geopandas.points_from_xy(toto.fac_lon, toto.fac_lat)
geo_df_list = [[point.xy[0][0], point.xy[1][0]] for point in toto.fac_geo]
print(geo_df_list)

facdest=geo_df_list[0]



[[31.725559186069926, 30.146057856944388]]


In [20]:
from xml.etree.ElementTree import tostring
import requests
import json

In [21]:



body = {"locations":[[31.625347137451172,30.087662159514313],facdest,[31.125184898206078,30.545245933532715]],"destinations":[0],"metrics":["duration","distance"],"sources":[1,2],"units":"m"}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf62487ad6039a4daa436d88a8a4ff481a40f9',
    'Content-Type': 'application/json; charset=utf-8'
}
call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)



print(call.status_code, call.reason)
print(call.text)
res = json.loads(call.text)
x=res['sources']
z=[]
for p in x:
    y=p['snapped_distance']
    z.append(y)
print(z)
z.sort()
m=z[0]

for i in x:
    if i['snapped_distance']==m:
        k=i['location']

print(res)
print('----------------------------')
print('closest facility:')        
print(k)

h=[31.625347137451172,30.087662159514313]
q=k[0]
w=k[1]

r=h[0]
t=h[1]
print(q)




200 OK
{"durations":[[1021.16],[3999.22]],"distances":[[18301.85],[89791.92]],"destinations":[{"location":[31.625268,30.087626],"snapped_distance":8.6}],"sources":[{"location":[31.725599,30.145981],"snapped_distance":9.4},{"location":[31.124768,30.545244],"snapped_distance":39.92}],"metadata":{"attribution":"openrouteservice.org | OpenStreetMap contributors","service":"matrix","timestamp":1647541138983,"query":{"locations":[[31.625347137451172,30.087662159514313],[31.725559186069926,30.146057856944388],[31.12518489820608,30.545245933532716]],"profile":"driving-car","responseType":"json","metricsStrings":["DISTANCE","DURATION"],"sources":["1","2"],"destinations":["0"],"metrics":["duration","distance"],"units":"m"},"engine":{"version":"6.7.0","build_date":"2022-02-18T19:37:41Z","graph_date":"2022-02-21T14:34:17Z"}}}
[9.4, 39.92]
{'durations': [[1021.16], [3999.22]], 'distances': [[18301.85], [89791.92]], 'destinations': [{'location': [31.625268, 30.087626], 'snapped_distance': 8.6}], 'so

In [22]:
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
}
call = requests.get('https://api.openrouteservice.org/v2/directions/driving-car?api_key=5b3ce3597851110001cf62487ad6039a4daa436d88a8a4ff481a40f9&start={},{}&end={},{}'.format(q,w,r,t), headers=headers)

print(call.status_code, call.reason)
print(call.text)


200 OK
{"type":"FeatureCollection","features":[{"bbox":[31.605338,30.087626,31.725757,30.145981],"type":"Feature","properties":{"segments":[{"distance":18301.9,"duration":1021.2,"steps":[{"distance":23.1,"duration":8.3,"type":11,"instruction":"Head southeast","name":"-","way_points":[0,1]},{"distance":262.3,"duration":47.2,"type":3,"instruction":"Turn sharp right onto شارع المشير احمد بدوى الجانبى","name":"شارع المشير احمد بدوى الجانبى","way_points":[1,8]},{"distance":10.5,"duration":3.8,"type":0,"instruction":"Turn left","name":"-","way_points":[8,9]},{"distance":674.3,"duration":48.5,"type":1,"instruction":"Turn right onto شارع المشير احمد بدوى","name":"شارع المشير احمد بدوى","way_points":[9,21]},{"distance":1468.8,"duration":105.7,"type":7,"instruction":"Enter the roundabout and take the 2nd exit onto طريق الحريه","name":"طريق الحريه","exit_number":2,"way_points":[21,56]},{"distance":842.3,"duration":50.5,"type":7,"instruction":"Enter the roundabout and take the 1st exit onto طريق ا